In [1]:
import findspark

In [2]:
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import  *

In [4]:
spark = SparkSession.builder \
.appName("DataframeWriter") \
.master("yarn") \
.enableHiveSupport() \
.getOrCreate()

In [5]:
df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.option("compression","gzip") \
.load("/user/train/datasets/Hotel_Reviews.csv.gz")

In [6]:
df2 = df.withColumn("Tags", 
                     F.split(F.col("Tags"), ",")
                     .cast(ArrayType(StringType()))) \
.withColumn("Review_Date", F.to_date(F.col("Review_Date"),"M/d/yyyy"))

In [7]:
df2.limit(2).toPandas()

Hotel_Address  \
0   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
1   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   

   Additional_Number_of_Scoring Review_Date  Average_Score   Hotel_Name  \
0                           194  2017-08-03            7.7  Hotel Arena   
1                           194  2017-08-03            7.7  Hotel Arena   

  Reviewer_Nationality                                    Negative_Review  \
0              Russia    I am so angry that i made this post available...   
1             Ireland                                         No Negative   

   Review_Total_Negative_Word_Counts  Total_Number_of_Reviews  \
0                                397                     1403   
1                                  0                     1403   

                                     Positive_Review  \
0   Only the park outside of the hotel was beauti...   
1   No real complaints the hotel was great great ...   

   Review_Total_Positive_Word_Counts  \
0                                 11   
1                                105   

   Total_Number_of_Reviews_Reviewer_Has_Given  Reviewer_Score  \
0                                           7             2.9   
1                                           7             7.5   

                                                Tags days_since_review  \
0  [[' Leisure trip ',  ' Couple ',  ' Duplex Dou...            0 days   
1  [[' Leisure trip ',  ' Couple ',  ' Duplex Dou...            0 days   

          lat        lng  
0  52.3605759  4.9159683  
1  52.3605759  4.9159683

In [8]:
df2.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [9]:
import time

# parquet with default snappy

In [10]:

start_time = time.time()

df2.write \
.format("parquet") \
.mode("overwrite") \
.save("hdfs://localhost:9000/user/train/output_data/hotel_review_parquet")


print("----------- %s secs ----------" %(time.time() - start_time))

----------- 32.173180103302 secs ----------


In [11]:
! hdfs dfs -ls -h /user/train/output_data/hotel_review_parquet

Found 2 items
-rw-r--r--   1 train supergroup          0 2022-01-02 00:14 /user/train/output_data/hotel_review_parquet/_SUCCESS
-rw-r--r--   1 train supergroup     57.5 M 2022-01-02 00:14 /user/train/output_data/hotel_review_parquet/part-00000-a0d30f7f-61e3-4976-b8aa-a84468905a5e-c000.snappy.parquet


# Parquet with gzip compression

In [12]:
start_time = time.time()

df2.write \
.format("parquet") \
.mode("overwrite") \
.option("compression","gzip") \
.save("hdfs://localhost:9000/user/train/output_data/hotel_review_parquet_gzip")


print("----------- %s secs ----------" %(time.time() - start_time))

----------- 47.52744722366333 secs ----------


In [14]:
! hdfs dfs -ls -h /user/train/output_data/hotel_review_parquet_gzip

Found 2 items
-rw-r--r--   1 train supergroup          0 2022-01-02 00:15 /user/train/output_data/hotel_review_parquet_gzip/_SUCCESS
-rw-r--r--   1 train supergroup     36.0 M 2022-01-02 00:15 /user/train/output_data/hotel_review_parquet_gzip/part-00000-2dfcc461-a24d-45e7-be91-2f570788a1b7-c000.gz.parquet


In [15]:
spark.stop()